# Multi-Microgrid Controllers

To actively manipulate multiple microgrids during experiments, we can implement a custom `Controller` that manages several microgrids simultaneously. 

In this example, our controller will coordinate two datacenters by adjusting their power consumption based on their current power deltas and battery states.

In [1]:
import pandas as pd
import vessim as vs

# Required for running Mosaic in Jupyter notebooks (fixes asyncio event loop conflicts)
import nest_asyncio
nest_asyncio.apply()

The key is implementing the `step()` method, which receives microgrid states as a dictionary. 
This allows the controller to coordinate across multiple microgrids.

In [2]:
from datetime import datetime

class CustomController(vs.Controller):
    def step(self, time: datetime, microgrid_states: dict[str, dict]) -> None:
        berlin = microgrid_states["Berlin"]
        mumbai = microgrid_states["Mumbai"]

        print(f"{time.strftime('%H:%M')}: Berlin: {berlin['p_delta']:.0f}W, Mumbai: {mumbai['p_delta']:.0f}W")

Now we'll create two datacenters in different cities and add our controller to monitor them.

In [5]:
environment = vs.Environment(sim_start="2022-06-15", step_size=300)

# Berlin datacenter
berlin = environment.add_microgrid(
    name="Berlin",
    actors=[
        vs.Actor(name="server", signal=vs.StaticSignal(value=-800)),
        vs.Actor(name="solar", signal=vs.Trace.load("solcast2022_global", column="Berlin", params={"scale": 2000})),
    ],
    storage=vs.SimpleBattery(capacity=700, initial_soc=0.7),
)

# Mumbai datacenter  
mumbai = environment.add_microgrid(
    name="Mumbai",
    actors=[
        vs.Actor(name="server", signal=vs.StaticSignal(value=-700)),
        vs.Actor(name="solar", signal=vs.Trace.load("solcast2022_global", column="Mumbai", params={"scale": 1800})),
    ],
    storage=vs.SimpleBattery(capacity=500),
)

# Add controllers
monitor = vs.Monitor([berlin, mumbai], outfile="./results.csv")
load_balancer = CustomController([berlin, mumbai])

environment.add_controller(monitor)
environment.add_controller(load_balancer)

environment.run(until=12 * 3600)  # 12 hours

2025-07-31 16:39:28.576 | INFO     | mosaik.async_scenario:start:420 - Starting "Actor" as "Berlin.actor.server" ...
2025-07-31 16:39:28.577 | INFO     | mosaik.async_scenario:start:420 - Starting "Actor" as "Berlin.actor.solar" ...
2025-07-31 16:39:28.578 | INFO     | mosaik.async_scenario:start:420 - Starting "Grid" as "Berlin.grid" ...
2025-07-31 16:39:28.579 | INFO     | mosaik.async_scenario:start:420 - Starting "Storage" as "Berlin.storage" ...
2025-07-31 16:39:28.690 | INFO     | mosaik.async_scenario:start:420 - Starting "Actor" as "Mumbai.actor.server" ...
2025-07-31 16:39:28.691 | INFO     | mosaik.async_scenario:start:420 - Starting "Actor" as "Mumbai.actor.solar" ...
2025-07-31 16:39:28.692 | INFO     | mosaik.async_scenario:start:420 - Starting "Grid" as "Mumbai.grid" ...
2025-07-31 16:39:28.693 | INFO     | mosaik.async_scenario:start:420 - Starting "Storage" as "Mumbai.storage" ...
2025-07-31 16:39:28.694 | INFO     | mosaik.async_scenario:start:420 - Starting "Controlle

00:00: Berlin: -800W, Mumbai: -700W
00:05: Berlin: -800W, Mumbai: -700W
00:10: Berlin: -800W, Mumbai: -700W
00:15: Berlin: -800W, Mumbai: -700W
00:20: Berlin: -800W, Mumbai: -700W
00:25: Berlin: -800W, Mumbai: -700W
00:30: Berlin: -800W, Mumbai: -700W
00:35: Berlin: -800W, Mumbai: -700W
00:40: Berlin: -800W, Mumbai: -700W
00:45: Berlin: -800W, Mumbai: -696W
00:50: Berlin: -800W, Mumbai: -691W
00:55: Berlin: -800W, Mumbai: -685W
01:00: Berlin: -800W, Mumbai: -676W
01:05: Berlin: -800W, Mumbai: -666W
01:10: Berlin: -800W, Mumbai: -654W
01:15: Berlin: -800W, Mumbai: -640W
01:20: Berlin: -800W, Mumbai: -627W
01:25: Berlin: -800W, Mumbai: -617W
01:30: Berlin: -800W, Mumbai: -610W
01:35: Berlin: -800W, Mumbai: -603W
01:40: Berlin: -800W, Mumbai: -592W
01:45: Berlin: -800W, Mumbai: -577W
01:50: Berlin: -800W, Mumbai: -558W
01:55: Berlin: -800W, Mumbai: -534W
02:00: Berlin: -800W, Mumbai: -508W
02:05: Berlin: -800W, Mumbai: -480W
02:10: Berlin: -800W, Mumbai: -450W
02:15: Berlin: -800W, Mumbai

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 43200/43200 [00:00<00:00, 123928.62steps/s]
2025-07-31 16:39:29.066 | INFO     | mosaik.async_scenario:run:845 - Simulation finished successfully.


07:15: Berlin: 69W, Mumbai: 355W
07:20: Berlin: 57W, Mumbai: 343W
07:25: Berlin: 66W, Mumbai: 341W
07:30: Berlin: 90W, Mumbai: 349W
07:35: Berlin: 122W, Mumbai: 361W
07:40: Berlin: 156W, Mumbai: 373W
07:45: Berlin: 192W, Mumbai: 386W
07:50: Berlin: 242W, Mumbai: 408W
07:55: Berlin: 304W, Mumbai: 444W
08:00: Berlin: 375W, Mumbai: 492W
08:05: Berlin: 440W, Mumbai: 521W
08:10: Berlin: 485W, Mumbai: 502W
08:15: Berlin: 515W, Mumbai: 438W
08:20: Berlin: 541W, Mumbai: 381W
08:25: Berlin: 564W, Mumbai: 374W
08:30: Berlin: 585W, Mumbai: 416W
08:35: Berlin: 606W, Mumbai: 471W
08:40: Berlin: 628W, Mumbai: 501W
08:45: Berlin: 649W, Mumbai: 503W
08:50: Berlin: 669W, Mumbai: 494W
08:55: Berlin: 686W, Mumbai: 481W
09:00: Berlin: 701W, Mumbai: 466W
09:05: Berlin: 717W, Mumbai: 431W
09:10: Berlin: 732W, Mumbai: 368W
09:15: Berlin: 748W, Mumbai: 278W
09:20: Berlin: 763W, Mumbai: 192W
09:25: Berlin: 776W, Mumbai: 138W
09:30: Berlin: 788W, Mumbai: 115W
09:35: Berlin: 800W, Mumbai: 106W
09:40: Berlin: 813

In [ ]:
berlin_df = pd.read_csv("results/Berlin.csv", index_col=0)
vs.plot_microgrid_trace(berlin_df)

In [ ]:
mumbai_df = pd.read_csv("results/Mumbai.csv", index_col=0)
vs.plot_microgrid_trace(mumbai_df)